In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np

In [2]:
# baca file train dan test
train_lines = []
test_lines = []
with open('kalimat_POSTag.txt', 'r') as f:
    train_lines = f.readlines()
with open('kalimat_POS_NE_test.txt', 'r') as f:
    test_lines = f.readlines()

In [3]:
# fungsi untuk konversi label NE
def convert_label(raw_ne):
    new_label = 0
    if (raw_ne=='B-PER'): new_label = 1
    if (raw_ne=='I-PER'): new_label = 2
    if (raw_ne=='B-ORG'): new_label = 3
    if (raw_ne=='I-ORG'): new_label = 4
    if (raw_ne=='B-LOC'): new_label = 5
    if (raw_ne=='I-LOC'): new_label = 6
    return new_label

In [4]:
# inisialisasi kode token dan postag
token_dict = {}
postag_dict = {}

#added prefix dict
prefix1_dict = {}
prefix2_dict = {}
prefix3_dict = {}

#added suffix dict
suffix1_dict = {}
suffix2_dict = {}
suffix3_dict = {}

#added postag dict
postag_dict = {}

counter_token = 0
counter_postag = 0

#added counter
counter_prefix1 = 0
counter_prefix2 = 0
counter_prefix3 = 0
counter_suffix1 = 0
counter_suffix2 = 0
counter_suffix3 = 0

In [5]:
# baca data token, postag, dan label NE
train_sents = []
test_sents = []
sent = []
counter = 0

In [6]:
# data train
for line in train_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        line_part.append('O')
        t = (line_part[0], line_part[1], convert_label(line_part[2]))
        if line_part[0].lower() not in token_dict.keys():
            token_dict[line_part[0].lower()] = counter_token
            counter_token = counter_token+1
            
            if len(line_part[0].lower())>=3:
                #prefix
                if line_part[0].lower()[0] not in prefix1_dict.keys():
                    prefix1_dict[line_part[0].lower()[0]] = counter_prefix1
                    counter_prefix1 = counter_prefix1+1
                if line_part[0].lower()[0:2] not in prefix2_dict.keys():
                    prefix2_dict[line_part[0].lower()[0:2]] = counter_prefix2
                    counter_prefix2 = counter_prefix2+1
                if line_part[0].lower()[0:3] not in prefix3_dict.keys():
                    prefix3_dict[line_part[0].lower()[0:3]] = counter_prefix3
                    counter_prefix3 = counter_prefix3+1
            
                #suffix
                if line_part[0].lower()[-1] not in suffix1_dict.keys():
                    suffix1_dict[line_part[0].lower()[-1]] = counter_suffix1
                    counter_suffix1 = counter_suffix1+1
                if line_part[0].lower()[-2:] not in suffix2_dict.keys():
                    suffix2_dict[line_part[0].lower()[-2:]] = counter_suffix2
                    counter_suffix2 = counter_suffix2+1
                if line_part[0].lower()[-3:] not in suffix3_dict.keys():
                    suffix3_dict[line_part[0].lower()[-3:]] = counter_suffix3
                    counter_suffix3 = counter_suffix3+1
            
            elif len(line_part[0].lower())>=2:
                #prefix
                if line_part[0].lower()[0] not in prefix1_dict.keys():
                    prefix1_dict[line_part[0].lower()[0]] = counter_prefix1
                    counter_prefix1 = counter_prefix1+1
                if line_part[0].lower()[0:2] not in prefix2_dict.keys():
                    prefix2_dict[line_part[0].lower()[0:2]] = counter_prefix2
                    counter_prefix2 = counter_prefix2+1
                prefix3_dict["novalues"] = 9999
                
                #suffix
                if line_part[0].lower()[-1] not in suffix1_dict.keys():
                    suffix1_dict[line_part[0].lower()[-1]] = counter_suffix1
                    counter_suffix1 = counter_suffix1+1
                if line_part[0].lower()[-2:] not in suffix2_dict.keys():
                    suffix2_dict[line_part[0].lower()[-2:]] = counter_suffix2
                    counter_suffix2 = counter_suffix2+1
                suffix3_dict["novalues"] = 9999
            else:
                #prefix
                if line_part[0].lower()[0] not in prefix1_dict.keys():
                    prefix1_dict[line_part[0].lower()[0]] = counter_prefix1
                    counter_prefix1 = counter_prefix1+1
                prefix2_dict["novalues"] = 9999
                prefix3_dict["novalues"] = 9999
                #suffix
                if line_part[0].lower()[-1] not in suffix1_dict.keys():
                    suffix1_dict[line_part[0].lower()[-1]] = counter_suffix1
                    counter_suffix1 = counter_suffix1+1
                suffix2_dict["novalues"] = 9999
                suffix3_dict["novalues"] = 9999
        if line_part[1] not in postag_dict.keys():
            postag_dict[line_part[1]] = counter_postag
            counter_postag = counter_postag+1
        #print(t)
        sent.append(t)
    else:
        print("train sent = ")
        print(sent)
        train_sents.append(sent)
        sent = []
        counter = counter+1

train sent = 
[('Pengamat', 'NN', 0), ('politik', 'NN', 0), ('dari', 'IN', 0), ('Universitas', 'NN', 0), ('Gadjah', 'NNP', 0), ('Mada', 'NNP', 0), (',', ',', 0), ('Arie', 'NNP', 0), ('Sudjito', 'NNP', 0), (',', ',', 0), ('menilai', 'VBT', 0), (',', ',', 0), ('keinginan', 'NN', 0), ('Ketua', 'NN', 0), ('Umum', 'JJ', 0), ('Partai', 'NN', 0), ('Golkar', 'NNP', 0), ('Aburizal', 'NNP', 0), ('Bakrie', 'NNP', 0), ('untuk', 'IN', 0), ('maju', 'VBI', 0), ('kembali', 'RB', 0), ('sebagai', 'IN', 0), ('ketua', 'NN', 0), ('umum', 'JJ', 0), ('merupakan', 'VBT', 0), ('pemaksaan', 'NN', 0), ('kehendak', 'NN', 0), ('.', '.', 0)]
train sent = 
[('Menurut', 'VBT', 0), ('dia', 'PRP', 0), (',', ',', 0), ('ada', 'VBI', 0), ('kesan', 'NN', 0), ('bahwa', 'SC', 0), ('Aburizal', 'NNP', 0), ('menggunakan', 'VBT', 0), ('segala', 'RB', 0), ('cara', 'NN', 0), ('untuk', 'IN', 0), ('memuluskan', 'VBT', 0), ('jalannya', 'NN', 0), ('kembali', 'RB', 0), ('menduduki', 'VBT', 0), ('Golkar', 'NNP', 0), ('1', 'CDP', 0), ('.

In [7]:
# data test
counter = 0
for line in test_lines:
    line = line.rstrip('\n')
    curr_tuple = ()
    if len(line)>1:
        line_part = line.split(" ")
        t = (line_part[0], line_part[1], convert_label(line_part[2]))
        #print(t)
        sent.append(t)
    else:
        print("test sent = ")
        print(sent)
        test_sents.append(sent)
        sent = []
        counter = counter+1

test sent = 
[('Menteri', 'NNP', 0), ('Energi', 'NNP', 0), ('dan', 'CC', 0), ('Sumber', 'NN', 0), ('Daya', 'NN', 0), ('Mineral', 'NNP', 0), ('(', 'OP', 0), ('ESDM', 'NNP', 0), (')', 'CP', 0), ('Arifin', 'NNP', 1), ('Tasrif', 'NNP', 2), ('meresmikan', 'VBT', 0), ('survei', 'NN', 0), ('seismik', 'NN', 0), ('untuk', 'IN', 0), ('mencari', 'VBT', 0), ('cadangan', 'NN', 0), ('minyak', 'NN', 0), ('dan', 'CC', 0), ('gas', 'NN', 0), ('(', 'OP', 0), ('migas)', 'NN', 0), ('.', '.', 0)]
test sent = 
[('Menteri', 'NN', 0), ('PPN/Kepala', 'Kepala', 0), ('Bappenas', 'NNP', 0), (',', ',', 0), ('Bambang', 'NNP', 0), ('Brodjonegoro', 'NNP', 0), ('meresmikan', 'VBT', 0), ('Peluncuran', 'NN', 0), ('Sekolah', 'NN', 0), ('Terintegrasi', 'NN', 0), ('Berpola', 'NN', 0), ('Asrama', 'NN', 0), ('dan', 'CC', 0), ('Demonstrasi', 'NN', 0), ('.', '.', 0)]
test sent = 
[('Menteri', 'NN', 0), ('ESDM', 'NNP', 0), ('Ignasius', 'NNP', 1), ('Jonan', 'NNP', 2), ('bersama', 'IN', 0), ('Gubernur', 'NN', 0), ('Kalimantan', 'N

In [8]:
# kode untuk token/kata dan postag yang tidak muncul di data training, namun muncul di data testing
token_dict['unk'] = 9999
postag_dict['unk'] = 9999
prefix1_dict['novalues'] = 9999
prefix2_dict['novalues'] = 9999
prefix3_dict['novalues'] = 9999
suffix1_dict['novalues'] = 9999
suffix2_dict['novalues'] = 9999
suffix3_dict['novalues'] = 9999

In [9]:
# fungsi untuk ekstraksi fitur dari sebuah kalimat
def word2features(sent, i): 
    prefix1 = "novalues"
    prefix2 = "novalues"
    prefix3 = "novalues"
    suffix1 = "novalues"
    suffix2 = "novalues"
    suffix3 = "novalues"
    word = sent[i][0]
    postag = sent[i][1]
    if word.lower() not in token_dict.keys(): 
        word = 'unk'
    if postag not in postag_dict.keys():
        postag = 'unk'
    if len(word) >= 3:
        if word[0] not in prefix1_dict.keys():
            prefix1 = "novalues"
        else:
            prefix1 = word[0]
        if word[0:2] not in prefix2_dict.keys():
            prefix2 = "novalues"
        else:
            prefix2 = word[0:2]
        if word[0:3] not in prefix3_dict.keys():
            prefix3 = "novalues"
        else:
            prefix3 = word[0:3]
        if word[-1] not in suffix1_dict.keys():
            suffix1 = "novalues"
        else:
            suffix1 = word[-1]
        if word[-2:] not in suffix2_dict.keys():
            suffix2 = "novalues"
        else:
            suffix2 = word[-2:]
        if word[-3:] not in suffix3_dict.keys():
            suffix3 = "novalues"
        else:
            suffix3 = word[-3:]
    elif len(word) >=2:
        if word[0] not in prefix1_dict.keys():
            prefix1 = "novalues"
        else:
            prefix1 = word[0]
        if word[0:2] not in prefix2_dict.keys():
            prefix2 = "novalues"
        else:
            prefix2 = word[0:2]
        prefix3 = "novalues"
        if word[-1] not in suffix1_dict.keys():
            suffix1 = "novalues"
        else:
            suffix1 = word[-1]
        if word[-2:] not in suffix2_dict.keys():
            suffix2 = "novalues"
        else:
            suffix2 = word[-2:]
        suffix3 = "novalues"
    else:
        if word[0] not in prefix1_dict.keys():
            prefix1 = "novalues"
        else:
            prefix1 = word[0]
            prefix2 = "novalues"
            prefix3 = "novalues"
        if word[-1] not in suffix1_dict.keys():
            suffix1 = "novalues"
        else:
            suffix1 = word[-1]
            suffix2 = "novalues"
            suffix3 = "novalues"
    features = [
        token_dict[word.lower()], # fitur kata dalam bentuk lowercase
        word.isupper(), # fitur apakah karakter pertama token merupakan huruf kapital
        word.istitle(), # fitur apakah token merupakan title
        word.isdigit(), # fitur apakah token merupakan digit
        prefix1_dict[prefix1], #fitur 1 karakter pertama token
        prefix2_dict[prefix2], #fitur 2 karakter pertama token
        prefix3_dict[prefix3], #fitur 3 karakter pertama token
        suffix1_dict[suffix1], #fitur 1 karakter terakhir token
        suffix2_dict[suffix2], #fitur 2 karakter terakhir token
        suffix3_dict[suffix3], #fitur 3 karakter terakhir token
        postag_dict[postag] # fitur kode postag token
    ]
                
    return features

In [10]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [11]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

In [12]:
def sent2tokens(sent):
    return [token for token, postag, label in sent]  

In [13]:
print('postag dictonary')
print(postag_dict)
print('token dictonary')
print(token_dict)

postag dictonary
{'NN': 0, 'IN': 1, 'NNP': 2, ',': 3, 'VBT': 4, 'JJ': 5, 'VBI': 6, 'RB': 7, '.': 8, 'PRP': 9, 'SC': 10, 'CDP': 11, 'DT': 12, '"': 13, 'MD': 14, 'CC': 15, 'OP': 16, '12': 17, 'CDI': 18, 'NEG': 19, 'FW': 20, 'RP': 21, '-': 22, 'WP': 23, '3': 24, 'NNG': 25, 'sama': 26, 'CP': 27, 'CDO': 28, '2': 29, '1': 30, '10': 31, 'CDC': 32, 'unk': 9999}
token dictonary
{'pengamat': 0, 'politik': 1, 'dari': 2, 'universitas': 3, 'gadjah': 4, 'mada': 5, ',': 6, 'arie': 7, 'sudjito': 8, 'menilai': 9, 'keinginan': 10, 'ketua': 11, 'umum': 12, 'partai': 13, 'golkar': 14, 'aburizal': 15, 'bakrie': 16, 'untuk': 17, 'maju': 18, 'kembali': 19, 'sebagai': 20, 'merupakan': 21, 'pemaksaan': 22, 'kehendak': 23, '.': 24, 'menurut': 25, 'dia': 26, 'ada': 27, 'kesan': 28, 'bahwa': 29, 'menggunakan': 30, 'segala': 31, 'cara': 32, 'memuluskan': 33, 'jalannya': 34, 'menduduki': 35, '1': 36, 'hal': 37, 'ini': 38, 'kata': 39, 'berpotensi': 40, 'menimbukan': 41, 'perpecahan': 42, 'di': 43, 'kalangan': 44, 'k

In [14]:
# ekstraksi fitur data train
X_train = []
y_train = []
for s in train_sents:
    for i in range(len(s)):
        X_train.append(word2features(s,i))
        y_train.append(s[i][2])

In [15]:
# ekstraksi fitur data test
X_test = []
y_test = []
for s in test_sents:
    for i in range(len(s)):
        X_test.append(word2features(s,i))
        y_test.append(s[i][2])

In [16]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [17]:
# train classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Coba test, satu kata
print('xtest')
print(X_test[0])

print(clf.predict(X_test[0].reshape(1,-1)))

# Coba test, keseluruhan data test
print('hasil klasifikasi data test:')
print(clf.predict(X_test))

xtest
[ 421    0    1    0 9999 9999 9999    2    2  137    2]
[0]
hasil klasifikasi data test:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


C:\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


In [18]:
print(classification_report(y_test,clf.predict(X_test)))
print("akurasi : ", round(clf.score(X_test,y_test)*100,2), "%")

              precision    recall  f1-score   support

           0       0.78      1.00      0.88        73
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00         1

   micro avg       0.78      0.78      0.78        93
   macro avg       0.13      0.17      0.15        93
weighted avg       0.62      0.78      0.69        93

akurasi :  78.49 %


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
